In [3]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [9]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    #build character vocabulary
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [10]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [12]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [13]:
#build dictionnary
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [17]:
#initialize tensors
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [18]:
#set tensor values
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [19]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [20]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100


  64/8000 [..............................] - ETA: 4:14 - loss: 0.0487

 128/8000 [..............................] - ETA: 2:27 - loss: 0.0745

 192/8000 [..............................] - ETA: 1:50 - loss: 0.0753

 256/8000 [..............................] - ETA: 1:31 - loss: 0.0757

...

7808/8000 [============================>.] - ETA: 0s - loss: 0.0281

7872/8000 [============================>.] - ETA: 0s - loss: 0.0281

7936/8000 [============================>.] - ETA: 0s - loss: 0.0281

8000/8000 [==============================] - 41s 5ms/step - loss: 0.0282 - val_loss: 0.9072


/Users/Cyril/PycharmProjects/NLP-experiment-1/virtualenv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [31]:

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [32]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [35]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !



-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: I see.
Decoded sentence: Je comprends.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !



-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: Oh no!
Decoded sentence: Oh non !

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Cheers!
Decoded sentence: Tchin-tchin !

-
Input sentence: Cheers!
Decoded sentence: Tchin-tchin !

-
Input sentence: Cheers!
Decoded sentence: Tchin-tchin !

-
Input sentence: Cheers!
Decoded sentence: Tchin-tchin !

-
Input sentence: Get up.
Decoded sentence: Lève-toi.

-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.



-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.

-
Input sentence: Go now.
Decoded sentence: Vas-y maintenant.

-
Input sentence: Got it!
Decoded sentence: Compris !

-
Input sentence: Got it!
Decoded sentence: Compris !

-
Input sentence: Got it?
Decoded sentence: Compris ?

-
Input sentence: Got it?
Decoded sentence: Compris ?

-
Input sentence: Got it?
Decoded sentence: Compris ?

-
Input sentence: Hop in.
Decoded sentence: Montez.

-
Input sentence: Hop in.
Decoded sentence: Montez.

-
Input sentence: Hug me.
Decoded sentence: Serrez-moi dans vos bras !

-


Input sentence: Hug me.
Decoded sentence: Serrez-moi dans vos bras !

-
Input sentence: I fell.
Decoded sentence: Je suis tombé.

-
Input sentence: I fell.
Decoded sentence: Je suis tombé.

-
Input sentence: I know.
Decoded sentence: Je sais.

-
Input sentence: I left.
Decoded sentence: Je suis partie.

-
Input sentence: I left.
Decoded sentence: Je suis partie.

-
Input sentence: I lost.
Decoded sentence: J'ai perdu.

-
Input sentence: I'm 19.
Decoded sentence: J'ai 19 ans.



...

-
Input sentence: No way!
Decoded sentence: C'est hors de question !

-
Input sentence: Really?
Decoded sentence: Vrai ?

-
Input sentence: Really?
Decoded sentence: Vrai ?

-
Input sentence: Really?
Decoded sentence: Vrai ?

-
Input sentence: Thanks.
Decoded sentence: Merci !

-
Input sentence: We try.
Decoded sentence: On essaye.

-
Input sentence: We won.
Decoded sentence: Nous l'avons emporté.

-
Input sentence: We won.
Decoded sentence: Nous l'avons emporté.

-
Input sentence: We won.
Decoded sentence: Nous l'avons emporté.



-
Input sentence: We won.
Decoded sentence: Nous l'avons emporté.

-
Input sentence: Ask Tom.
Decoded sentence: Demande à Tom.

-
Input sentence: Awesome!
Decoded sentence: Fantastique !

-
Input sentence: Be calm.
Decoded sentence: Soyez calme !

-
Input sentence: Be calm.
Decoded sentence: Soyez calme !

-
Input sentence: Be calm.
Decoded sentence: Soyez calme !

-
Input sentence: Be cool.
Decoded sentence: Sois détendu !



-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be fair.
Decoded sentence: Soyez juste !

-
Input sentence: Be kind.
Decoded sentence: Sois gentil.

-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !

-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !



-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !

-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !

-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !

-
Input sentence: Be nice.
Decoded sentence: Soyez gentil !

-
Input sentence: Beat it.
Decoded sentence: Dégage !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

-
Input sentence: Come in.
Decoded sentence: Entre !



-
Input sentence: Come in.
Decoded sentence: Entre !

-
Input sentence: Come in.
Decoded sentence: Entre !

-
Input sentence: Come in.
Decoded sentence: Entre !

-
Input sentence: Come on!
Decoded sentence: Allez !

-
Input sentence: Come on.
Decoded sentence: Venez !

